In [229]:
import random
import time

class SmartComputerPlayer:
    #implemetning static method instead of instance for better performance
    @staticmethod
    def make_move(game):
        player_symbol = game.player_symbols[game.current_player]
        
        move = SmartComputerPlayer._try_to_complete_row(game, player_symbol)
        if move:
            return move
            
        opponent = 'Red' if game.current_player == 'Blue' else 'Blue'
        opponent_symbol = game.player_symbols[opponent]
        move = SmartComputerPlayer._try_to_block_opponent(game, opponent_symbol)
        if move:
            return move
            
        move = SmartComputerPlayer._make_strategic_move(game, player_symbol)
        if move:
            return move
            
        return SmartComputerPlayer._make_random_move(game, player_symbol)
    
    @staticmethod
    def _try_to_complete_row(game, symbol):
        for row in range(game.n):
            for col in range(game.n):
                if not game.board[row][col]:  # Empty cell
                    if SmartComputerPlayer._would_form_three_in_row(game, row, col, symbol):
                        return row, col, symbol
        
        return None
    
    @staticmethod
    def _would_form_three_in_row(game, row, col, symbol):
        directions = [
            (0, 1),
            (1, 0),
            (1, 1),
            (1, -1),
        ]
        
        for dr, dc in directions:
            count_pos = 0
            for i in range(1, 3):
                r, c = row + i*dr, col + i*dc
                if 0 <= r < game.n and 0 <= c < game.n:
                    cell = game.board[r][c]
                    if cell and cell[0] == symbol:
                        count_pos += 1
                    else:
                        break
                else:
                    break
                        
            count_neg = 0
            for i in range(1, 3):
                r, c = row - i*dr, col - i*dc
                if 0 <= r < game.n and 0 <= c < game.n:
                    cell = game.board[r][c]
                    if cell and cell[0] == symbol:
                        count_neg += 1
                    else:
                        break
                else:
                    break
            
            if count_pos + count_neg + 1 >= 3:
                return True
        
        return False
    
    @staticmethod
    def _try_to_block_opponent(game, opponent_symbol):
        for row in range(game.n):
            for col in range(game.n):
                if not game.board[row][col]:
                    if SmartComputerPlayer._would_form_three_in_row(game, row, col, opponent_symbol):
                        player_symbol = game.player_symbols[game.current_player]
                        return row, col, player_symbol
        
        return None
    
    @staticmethod
    def _make_strategic_move(game, symbol):
        empty_count = sum(1 for row in game.board for cell in row if not cell)
        if empty_count > game.n * game.n * 0.7:
            center = game.n // 2
            center_options = []
            
            if not game.board[center][center]:
                return center, center, symbol
                
            for r in range(center-1, center+2):
                for c in range(center-1, center+2):
                    if 0 <= r < game.n and 0 <= c < game.n and not game.board[r][c]:
                        center_options.append((r, c))
            
            if center_options:
                row, col = random.choice(center_options)
                return row, col, symbol
        
        strategic_spots = []
        for row in range(game.n):
            for col in range(game.n):
                if not game.board[row][col]:
                    score = SmartComputerPlayer._evaluate_position(game, row, col, symbol)
                    strategic_spots.append((score, row, col))
        
        if strategic_spots:
            strategic_spots.sort(reverse=True)
            top_spots = [spot for spot in strategic_spots if spot[0] >= strategic_spots[0][0] * 0.8]
            _, row, col = random.choice(top_spots)
            return row, col, symbol
        
        return None
    
    @staticmethod
    def _evaluate_position(game, row, col, symbol):
        score = 0
        directions = [
            (0, 1),
            (1, 0),
            (1, 1),
            (1, -1),
        ]
        
        for dr, dc in directions:

            same_symbols = 0
            empty_spaces = 0
            
            for i in range(1, 3):
                r, c = row + i*dr, col + i*dc
                if 0 <= r < game.n and 0 <= c < game.n:
                    cell = game.board[r][c]
                    if not cell:
                        empty_spaces += 1
                    elif cell[0] == symbol:
                        same_symbols += 1
                    else:
                        break
                else:
                    break
            
            for i in range(1, 3):
                r, c = row - i*dr, col - i*dc
                if 0 <= r < game.n and 0 <= c < game.n:
                    cell = game.board[r][c]
                    if not cell:
                        empty_spaces += 1
                    elif cell[0] == symbol:
                        same_symbols += 1
                    else:
                        break
                else:
                    break
            
            if same_symbols + empty_spaces >= 2:
                if same_symbols == 2:
                    score += 10
                elif same_symbols == 1: 
                    score += 5
                else: 
                    score += 1
        
        center = game.n // 2
        distance_from_center = max(abs(row-center), abs(col-center))
        center_score = max(0, game.n//2 - distance_from_center)
        score += center_score
        
        return score
    
    @staticmethod
    def _make_random_move(game, symbol):

        empty_cells = [(r, c) for r in range(game.n) for c in range(game.n) if not game.board[r][c]]
        if empty_cells:
            row, col = random.choice(empty_cells)
            return row, col, symbol
        return None


class BaseSOSGame:
    def __init__(self, n=8, player_type={'Blue': 'human', 'Red': 'human'}):
        self.n = n
        self.player_type = player_type
        self.board = [['' for _ in range(n)] for _ in range(n)]
        self.current_player = 'Blue'
        self.player_symbols = {'Blue': 'S', 'Red': 'O'}
        self.winner = None
        self.game_over = False

    def place_symbol(self, row=None, col=None):
        # If its a computer player turn
        if self.player_type[self.current_player] == 'computer':

            move = SmartComputerPlayer.make_move(self)
            if move:
                row, col, _ = move
                symbol = self.player_symbols[self.current_player]
                
                # Place the symbol
                if not self.game_over and 0 <= row < self.n and 0 <= col < self.n and self.board[row][col] == '':
                    self.board[row][col] = (symbol, self.current_player)
                    
                    self.check_score_or_win(row, col)
                    return True
            
            return False
        
        # Human player's turn
        elif row is not None and col is not None:
            if not self.game_over and 0 <= row < self.n and 0 <= col < self.n and self.board[row][col] == '':
                symbol = self.player_symbols[self.current_player]
                self.board[row][col] = (symbol, self.current_player)
                
                self.check_score_or_win(row, col)
                return True
        
        return False

    def switch_turn(self):
        self.current_player = 'Red' if self.current_player == 'Blue' else 'Blue'

    def swap_roles(self):
        
        self.player_symbols = {'Blue': 'O', 'Red': 'S'} if self.player_symbols['Blue'] == 'S' else {'Blue': 'S', 'Red': 'O'}

    def set_board_size(self, new_size):
        try:
            new_size = int(new_size)
            if new_size > 2:
                self.n = new_size
                self.board = [['' for _ in range(new_size)] for _ in range(new_size)]
                return True
        except ValueError:
            pass
        return False

    def reset_game(self):
        self.board = [['' for _ in range(self.n)] for _ in range(self.n)]
        self.current_player = 'Blue'
        self.winner = None
        self.game_over = False

    def check_score_or_win(self, row, col):

        self.switch_turn()

class SimpleSOSGame(BaseSOSGame):
    def check_score_or_win(self, row, col):
        
        current_player = self.board[row][col][1]
        
        # Check for SOS sequence
        directions = [
            (0, 1),
            (1, 0),
            (1, 1), 
            (1, -1),
        ]

        for dr, dc in directions:

            for offset in range(-2, 1):
                try:
                    r1 = row + offset * dr
                    c1 = col + offset * dc
                    r2 = r1 + dr
                    c2 = c1 + dc
                    r3 = r2 + dr
                    c3 = c2 + dc

                    if all(0 <= r < self.n and 0 <= c < self.n for r, c in [(r1, c1), (r2, c2), (r3, c3)]):
                        s1 = self.board[r1][c1]
                        s2 = self.board[r2][c2]
                        s3 = self.board[r3][c3]

                        if s1 and s2 and s3:
                            # Check for SOS pattern
                            if s1[0] == 'S' and s2[0] == 'O' and s3[0] == 'S':
                                self.winner = current_player
                                self.game_over = True
                                return
                except IndexError:
                    continue
        
        # Check if the board is full (end the game)
        if all(cell for row in self.board for cell in row):
            self.game_over = True
            self.winner = 'Draw'
            
        self.switch_turn()  # Switch turn


class GeneralSOSGame(BaseSOSGame):
    @property
    def scores_display(self):
        if self.game_over:
            if self.winner == 'Draw':
                return f"Final Score — Blue: {self.scores['Blue']}  Red: {self.scores['Red']} — It's a Draw!"
            else:
                return f"Final Score — Blue: {self.scores['Blue']}  Red: {self.scores['Red']} — {self.winner} Wins!"
        return f"Score — Blue: {self.scores['Blue']}  Red: {self.scores['Red']}"
        
    def __init__(self, n=8, player_type={'Blue': 'human', 'Red': 'human'}):
        super().__init__(n, player_type=player_type)
        self.scores = {'Blue': 0, 'Red': 0}

    def check_score_or_win(self, row, col):
        
        current_player = self.board[row][col][1]
        
        # Check for SOS sequence
        directions = [
            (0, 1),   # Horizontal
            (1, 0),   # Vertical
            (1, 1),   # Diagonal ↘
            (1, -1),  # Diagonal ↙
        ]

        made_point = False
        
        for dr, dc in directions:

            for offset in range(-2, 1):
                try:
                    r1 = row + offset * dr
                    c1 = col + offset * dc
                    r2 = r1 + dr
                    c2 = c1 + dc
                    r3 = r2 + dr
                    c3 = c2 + dc

                    if all(0 <= r < self.n and 0 <= c < self.n for r, c in [(r1, c1), (r2, c2), (r3, c3)]):
                        s1 = self.board[r1][c1]
                        s2 = self.board[r2][c2]
                        s3 = self.board[r3][c3]

                        if s1 and s2 and s3:
                            # Check for SOS pattern
                            if s1[0] == 'S' and s2[0] == 'O' and s3[0] == 'S':
                                self.scores[current_player] += 1
                                made_point = True
                except IndexError:
                    continue

        # End game when board is full
        if all(cell for row in self.board for cell in row):
            self.game_over = True
            if self.scores['Blue'] > self.scores['Red']:
                self.winner = 'Blue'
            elif self.scores['Red'] > self.scores['Blue']:
                self.winner = 'Red'
            else:
                self.winner = 'Draw'
            return
        
        if not made_point:
            # Only switch turn if no point was made
            self.switch_turn()


class SOSGame:
    def __init__(self, n=8, mode="Simple", player_type={'Blue': 'human', 'Red': 'human'}):
        self.player_type = player_type
        self.mode = mode
        self.n = n
        self._build_game()

    def _build_game(self):
        kwargs = {'n': self.n, 'player_type': self.player_type}
        if self.mode == "Simple":
            self.game_instance = SimpleSOSGame(**kwargs)
        else:
            self.game_instance = GeneralSOSGame(**kwargs)

    def set_game_mode(self, mode):
        self.mode = mode
        self._build_game()
        
    def update(self):
        """This is a stub that doesn't need to do anything"""
        pass

    def __getattr__(self, item):
        return getattr(self.game_instance, item)